In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
import os
os.chdir('F:\Machine Learning')

In [3]:
filename = "nnp.txt"
raw_text = open(filename).read()

In [4]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [5]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  3649
Total Vocab:  44


In [18]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 256
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  3393


In [19]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [22]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.35))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [23]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [24]:
model.fit(X, y, epochs=20, batch_size=256, callbacks=callbacks_list)

Epoch 1/20
3393/3393 [==============================] - 41s 12ms/step - loss: 3.3561

Epoch 00001: loss improved from inf to 3.35610, saving model to weights-improvement-01-3.3561.hdf5
Epoch 2/20
3393/3393 [==============================] - 44s 13ms/step - loss: 2.9913

Epoch 00002: loss improved from 3.35610 to 2.99126, saving model to weights-improvement-02-2.9913.hdf5
Epoch 3/20
3393/3393 [==============================] - 52s 15ms/step - loss: 2.9557

Epoch 00003: loss improved from 2.99126 to 2.95565, saving model to weights-improvement-03-2.9557.hdf5
Epoch 4/20
3393/3393 [==============================] - 57s 17ms/step - loss: 2.9321

Epoch 00004: loss improved from 2.95565 to 2.93206, saving model to weights-improvement-04-2.9321.hdf5
Epoch 5/20
3393/3393 [==============================] - 63s 18ms/step - loss: 2.9321

Epoch 00005: loss did not improve from 2.93206
Epoch 6/20
3393/3393 [==============================] - 58s 17ms/step - loss: 2.9221

Epoch 00006: loss improved fr

In [25]:
# load the network weights
filename = "weights-improvement-19-2.8489.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [26]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [27]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" 6|
"D7"A3 B c2|c3 B A2|"G"B4 c2|d6|"C"e4 e2|g3 f e2|
"G"d6|B6|"D7"A3 B c2|D2 E2 F2|"G"G3 A GF|!D.C.!G6|]
!mp! D2|:"G" G4 B2|d4 B2|"C"c3 e c2|"D"A4 F2|"G"G4 B2|d3 g g2|
"G"f d3 "Em"B2|"D"d4 c2|"G"B4 G2|D4 B,2|"C"E3 C E2|"D"D4 GF|
"C"E3 "G"D "C"E2|"Am"C "G"B "


NameError: name 'sys' is not defined